In [1]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
df = pd.read_csv('./traces.csv')
df.head()

,fno,xmin,ymin,xmax,ymax,confidence,cls
0,0,815.328613,414.294556,1023.535522,710.458862,0.840505,1.0
1,1,815.099548,416.698364,1024.056885,710.406616,0.839022,1.0
2,2,816.624023,416.424591,1025.664185,707.520020,0.802077,1.0
3,3,815.463745,414.316956,1025.545410,707.625366,0.820569,1.0
4,11,818.357239,411.877686,1025.415894,708.695923,0.819219,1.0


In [3]:
df['fno'].count()

438

In [4]:
df_conf_filtered = df[df['confidence'] > 0.7].reset_index(drop=True)
df_conf_filtered['fno'].count()

438

In [5]:
df_conf_filtered.dtypes

fno             int64
xmin          float64
ymin          float64
xmax          float64
ymax          float64
confidence    float64
cls           float64
dtype: object

In [6]:
df_conf_filtered.head()

,fno,xmin,ymin,xmax,ymax,confidence,cls
0,0,815.328613,414.294556,1023.535522,710.458862,0.840505,1.0
1,1,815.099548,416.698364,1024.056885,710.406616,0.839022,1.0
2,2,816.624023,416.424591,1025.664185,707.520020,0.802077,1.0
3,3,815.463745,414.316956,1025.545410,707.625366,0.820569,1.0
4,11,818.357239,411.877686,1025.415894,708.695923,0.819219,1.0


In [7]:
cap = cv2.VideoCapture('../videos/capture.mp4')

In [8]:
def mark_frame(cap, fno, pos, cls):
    cap.set(cv2.CAP_PROP_POS_FRAMES, fno)
    res, frame = cap.read()
    if not res:
        return None

    print(pos[0:2])
    print(pos[2:4])

    img = cv2.rectangle(frame, pos[0:2], pos[2:4], (255, 0, 0), 6)
    img = cv2.putText(img, f'{cls}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    return img

def mark_and_show(cap, fno, pos, cls):
    img = mark_frame(cap, fno, pos, cls) 
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(rgb)
    plt.show()


In [9]:
record = df_conf_filtered.iloc[0, :].to_numpy().astype(np.int32)
record

array([   0,  815,  414, 1023,  710,    0,    1], dtype=int32)

In [10]:
marked = mark_frame(cap, record[0], record[1:5], record[5])

[815 414]
[1023  710]


In [11]:
# parameters
window_size = 5
with open('./window_size.txt', 'r') as f:
    s = f.read()
    window_size = int(s)

In [12]:
# prepare x axis
labels = df_conf_filtered.columns.drop(['fno', 'confidence'])
new_labels = []

for i in range(window_size):
    for label in labels:
        new_labels.append(f'{i}_{label}')

new_labels.append('waving')
print(new_labels)
print(len(new_labels))

['0_xmin', '0_ymin', '0_xmax', '0_ymax', '0_cls', '1_xmin', '1_ymin', '1_xmax', '1_ymax', '1_cls', '2_xmin', '2_ymin', '2_xmax', '2_ymax', '2_cls', '3_xmin', '3_ymin', '3_xmax', '3_ymax', '3_cls', '4_xmin', '4_ymin', '4_xmax', '4_ymax', '4_cls', '5_xmin', '5_ymin', '5_xmax', '5_ymax', '5_cls', '6_xmin', '6_ymin', '6_xmax', '6_ymax', '6_cls', '7_xmin', '7_ymin', '7_xmax', '7_ymax', '7_cls', '8_xmin', '8_ymin', '8_xmax', '8_ymax', '8_cls', '9_xmin', '9_ymin', '9_xmax', '9_ymax', '9_cls', 'waving']
51


In [13]:
import json

f = open('../data/timeseries/waving.json')
ts_result = json.load(f)
f.close()

In [14]:
results = ts_result[0]['annotations'][0]['result']
results

waving_ranges = []

for result in results:
    waving_ranges.append((result['value']['start'], result['value']['end']))

In [15]:
waving_ranges

[(68, 150), (320, 380), (490, 551)]

In [16]:
length = df_conf_filtered.shape[0]

target = pd.DataFrame([], columns=new_labels)
for i in range(0, length-window_size):
    sub_df = df_conf_filtered.iloc[i:i+window_size, :]    
    if sub_df.iloc[window_size-1]['fno'] - sub_df.iloc[0]['fno'] == window_size-1:
        waving = False
        for rg in waving_ranges:
            if sub_df.iloc[window_size-1]['fno'] <= rg[1] and sub_df.iloc[0]['fno'] >= rg[0]:
                waving = True
                break
        
        sub_df = sub_df.drop(['fno', 'confidence'], axis=1)
        sub_widen = np.concatenate([
            sub_df.to_numpy().reshape(window_size*len(sub_df.columns)),
            np.array([waving])
        ]).reshape(1, window_size*len(sub_df.columns) + 1)
        sub_df_widen = pd.DataFrame(sub_widen, columns=new_labels)

        target = pd.concat([target, sub_df_widen])

In [17]:
target

,0_xmin,0_ymin,0_xmax,0_ymax,0_cls,1_xmin,1_ymin,1_xmax,1_ymax,1_cls,...,8_ymin,8_xmax,8_ymax,8_cls,9_xmin,9_ymin,9_xmax,9_ymax,9_cls,waving
0,447.183441,367.721252,762.262268,609.651672,3.0,452.539215,329.056763,771.376282,595.739868,3.0,...,131.966110,855.432617,466.804382,3.0,546.723816,113.688324,863.511047,452.275787,3.0,0.0
0,452.539215,329.056763,771.376282,595.739868,3.0,469.208527,293.963135,792.578796,565.052612,3.0,...,113.688324,863.511047,452.275787,3.0,570.173767,106.929611,879.735474,438.451965,3.0,0.0
0,469.208527,293.963135,792.578796,565.052612,3.0,484.448669,253.992157,781.193726,541.653442,3.0,...,106.929611,879.735474,438.451965,3.0,585.968628,95.638176,886.465027,430.025482,3.0,0.0
0,484.448669,253.992157,781.193726,541.653442,3.0,488.186737,221.486725,798.628296,527.634033,3.0,...,95.638176,886.465027,430.025482,3.0,601.677063,82.272400,890.410217,429.059723,3.0,0.0
0,488.186737,221.486725,798.628296,527.634033,3.0,503.780029,201.353729,823.501465,515.097473,3.0,...,82.272400,890.410217,429.059723,3.0,598.257812,73.965874,894.807373,413.812225,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,699.461426,423.733612,957.452942,708.651062,1.0,699.406982,424.680389,961.169250,707.563049,1.0,...,400.851044,948.700256,704.219666,1.0,702.851440,399.100464,949.783997,700.597107,1.0,0.0
0,699.406982,424.680389,961.169250,707.563049,1.0,698.133545,420.932404,959.700623,707.883362,1.0,...,399.100464,949.783997,700.597107,1.0,701.266479,395.763855,949.154297,702.368164,1.0,0.0
0,698.133545,420.932404,959.700623,707.883362,1.0,701.597412,419.343658,957.017029,707.357666,1.0,...,395.763855,949.154297,702.368164,1.0,700.758301,395.922302,950.325012,699.133972,1.0,0.0
0,701.597412,419.343658,957.017029,707.357666,1.0,701.923828,416.039215,954.468933,706.537292,1.0,...,395.922302,950.325012,699.133972,1.0,699.762573,397.087250,951.128723,701.405823,1.0,0.0


In [18]:
target.to_csv('./traces_seq.csv', index=False)

In [19]:
with open('./window_size.txt', 'w') as f:
    f.write(f'{window_size}')

In [20]:
a = np.array([1, 2, 3])

In [21]:
np.concatenate([a, np.array([1])], axis=0)

array([1, 2, 3, 1])

In [22]:
sub_df = df_conf_filtered.iloc[0:5, :]
sub_df.iloc[4]['fno'] > waving_ranges[0][0]

False